In [ ]:
import sys
sys.path.insert(0, '..')

In [ ]:
from data import *
from datetime import datetime
from typing import Dict, List
import os
import pandas as pd
from pandas import DataFrame, Series, DatetimeIndex, Index
import wandb
import numpy as np
from numpy import ndarray
from tqdm import tqdm

In [ ]:
engine: Engine = db_connection()
sql_energy: str = "SELECT * FROM sorgenia_energy"
energy_df: DataFrame = pd.read_sql_query(sql_energy, con=engine)
energy_grouped: DataFrame = group_hourly(energy_df)

In [ ]:
energy_grouped.head()

In [ ]:
# pivot energy grouped
energy_grouped_piv = energy_grouped.pivot(index='time', columns='plant_name_up', values='kwh')
energy_grouped_piv.columns.name = None
energy_grouped_piv.reset_index(drop=False, inplace=True)

In [ ]:
energy_grouped_piv.head()

In [ ]:
for up in tqdm(energy_grouped.plant_name_up.unique()):
    up1: ndarray = energy_grouped['kwh'][energy_grouped['plant_name_up']=='UP_MPNTLCDMRN_1'].values
    up1_piv: ndarray = energy_grouped_piv['UP_MPNTLCDMRN_1'].values
    assert np.array_equal(up1, up1_piv)

In [ ]:
weather_df: DataFrame = pd.read_sql_query("SELECT * FROM sorgenia_weather_copernicus", con=engine)
weather_df['wind_gusts_100m_1h_ms'] = weather_df['wind_gusts_100m_1h_ms'].astype('float64')
weather_df['wind_gusts_100m_ms'] = weather_df['wind_gusts_100m_ms'].astype('float64')
weather_df: DataFrame = weather_df.sort_values(by=['timestamp_utc'], ascending=True, ignore_index=True)
# pivot weather df
weather_df_piv = weather_df.pivot(index='timestamp_utc', columns=['plant_name_up'], values=list(weather_df.columns[3:]))
new_index = Index([i[0]+'_'+i[1] for i in weather_df_piv.columns])
weather_df_piv.columns = new_index
weather_df_piv.reset_index(drop=False, inplace=True)

In [ ]:
weather_df_piv.head()

MERGE

In [ ]:
df: DataFrame = energy_grouped_piv.merge(weather_df_piv, left_on='time',
                                 right_on='timestamp_utc')
df.drop(['timestamp_utc'], axis=1, inplace=True)
df = df.sort_values(by=['time'], ascending=True, ignore_index=True)

In [ ]:
df.columns